In [1]:
from ptrail.core.Datasets import Datasets
import pandas as pd

In [2]:
geolife = Datasets.load_geo_life_sample()
geolife

------------------------ Dataset Facts ------------------------------

Number of unique Trajectories in the data: 2
Number of points in the data: 217653
Dataset time range: 146 days 23:53:26
Datatype of the DataFrame: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>
Dataset Bounding Box: (22.147577, 113.548843, 41.132062, 121.156224)

---------------------------------------------------------------------


lat         lon
traj_id DateTime                                  
1       2008-10-23 05:53:11  39.984224  116.319402
        2008-10-23 05:53:16  39.984211  116.319389
        2008-10-23 05:53:21  39.984217  116.319422
        2008-10-23 05:53:23  39.984710  116.319865
        2008-10-23 05:53:28  39.984674  116.319810
...                                ...         ...
5       2009-03-19 05:46:02  39.999896  116.327290
        2009-03-19 05:46:07  39.999899  116.327352
        2009-03-19 05:46:12  39.999945  116.327394
        2009-03-19 05:46:17  40.000015  116.327433
        2009-03-19 05:46:37  39.999978  116.327460

[217653 rows x 2 columns]